### Question

Arrays A and B consisting of N non-negative integers are given. Together, they represent N real numbers, denoted as C[0], ..., C[N−1]. Elements of A represent the integer parts and the corresponding elements of B (divided by 1,000,000) represent the fractional parts of the elements of C. More formally, A[I] and B[I] represent C[I] = A[I] + B[I] / 1,000,000.

For example, consider the following arrays A and B:

  A[0] = 0	B[0] = 500,000
  
  A[1] = 1	B[1] = 500,000
  
  A[2] = 2	B[2] = 0
  
  A[3] = 2      B[3] = 0
  
  A[4] = 3	B[4] = 0
  
  A[5] = 5	B[5] = 20,000

They represent the following real numbers:
  C[0] = 0.5
  C[1] = 1.5
  C[2] = 2.0
  C[3] = 2.0
  C[4] = 3.0
  C[5] = 5.02

A pair of indices (P, Q) is multiplicative if 0 ≤ P < Q < N and C[P] * C[Q] ≥ C[P] + C[Q].

The above arrays yield the following multiplicative pairs:

        (1, 4), because 1.5 * 3.0 = 4.5 ≥ 4.5 = 1.5 + 3.0,
        (1, 5), because 1.5 * 5.02 = 7.53 ≥ 6.52 = 1.5 + 5.02,
        (2, 3), because 2.0 * 2.0 = 4.0 ≥ 4.0 = 2.0 + 2.0,
        (2, 4) and (3, 4), because 2.0 * 3.0 = 6.0 ≥ 5.0 = 2.0 + 3.0.
        (2, 5) and (3, 5), because 2.0 * 5.02 = 10.04 ≥ 7.02 = 2.0 + 5.02.
        (4, 5), because 3.0 * 5.02 = 15.06 ≥ 8.02 = 3.0 + 5.02.

Write a function:

    def solution(A, B):

that, given arrays A and B, each containing N non-negative integers, returns the number of multiplicative pairs of indices.

If the number of multiplicative pairs is greater than 1,000,000,000, the function should return 1,000,000,000.

For example, given:

  A[0] = 0	B[0] = 500,000
  
  A[1] = 1	B[1] = 500,000
  
  A[2] = 2	B[2] = 0
  
  A[3] = 2	B[3] = 0
  
  A[4] = 3	B[4] = 0
  
  A[5] = 5	B[5] = 20,000

the function should return 8, as explained above.

Write an algorithm for the following assumptions:

        N is an integer within the range [0..100,000];
        each element of array A is an integer within the range [0..1,000];
        each element of array B is an integer within the range [0..999,999];
        real numbers created from arrays are sorted in non-decreasing order.

In [1]:
import numpy as np

Naive solution

In [2]:
def brute_force_solution(A, B):
    
    # Loop in Loop
   
    assert len(A) == len (B), 'A & B must have same number of elements'
    assert len(A) != 1      , 'number of elements must be > 1'

    ans = 0  # initialize

    C = np.array(A) + np.array(B)/1000000
    
    for i in range(len(A)):
        for j in range(i):
            if C[i]*C[j] >= C[i]+C[j]:
                #print(i,j,'-----',1,'-----',C[i],C[j])
                ans += 1
        #    else:
        #        print(i,j,'-----',0,'-----',C[i],C[j])
        #print('-----------')
        if ans >= 1000000000:
            return 1000000000
        
    return ans

We need to solve this equation:

`xy >= x+y` <br>
`(x-1)y >= x` <br>
This is a hyperbola with center (1,1) in xy plane, vertices (0,0) & (2,2), and asymptotes `x=1` and `y=1` 

we can see that:

    -If 0 ≤ C[P] < 1, then (a, b) is multiplicative only if C[P] = 0
    -If 1 < C[P] < 2, then C[Q] > 2. So, C[Q] > C[P], 
    but that contradicts C[P] ≥ C[Q] (given the array is sorted and function is monotonic).
    -If C[P] > 2 then the pair is multiplicative for any C[Q] where C[Q] ≥ C[P] / (C[P] - 1)


In [3]:
def hyperbola_solution(A,B):
    
    ans = 0  # initialize
    
    C = np.array(A) + np.array(B)/1000000
    
    # Add zero pairs
    zero_pairs = np.count_nonzero(C==0)
    ans = int(zero_pairs*(zero_pairs-1)/2)
           
    hi_index = len(C) - 1
    
    # Skip all C[i] less than 1
    for lo_index in range(hi_index):
        if (C[lo_index] > 1):
            break
            
    while (hi_index > lo_index):
        v = C[hi_index] / (C[hi_index] - 1)
        
        while (lo_index < hi_index and C[lo_index] < v):
            lo_index+=1
            
        if lo_index == hi_index:
            break
            
        ans += (hi_index - lo_index)
        
        if ans >= 1000000000:
            return 1000000000
        
        hi_index-=1
        
    return ans

In [4]:
hyperbola_solution(np.array([0,1,2,2,3,5]),[500000,500000,0,0,0,20000])

8

In [5]:
hyperbola_solution(np.array([0,0,0,1,2,2,3,5]),[0,0,500000,500000,0,0,0,20000])

9

In [6]:
brute_force_solution(np.array([0,0,0,1,2,2,3,5]),[0,0,500000,500000,0,0,0,20000])

9

Testing on a random array of size 500

In [7]:
A = np.random.randint(1000   , size=(500))
B = np.random.randint(1000000, size=(500))

C = A + B/1000000

C = np.sort(C)

A = C.astype(int)
B = C-A

In [8]:
brute_force_solution(A,B)

124251

In [9]:
hyperbola_solution(A,B)

124251

In [10]:
%timeit brute_force_solution(A,B)

81.9 ms ± 1.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit hyperbola_solution(A,B)

424 µs ± 1.28 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
A = np.random.randint(1000   , size=(300000))
B = np.random.randint(1000000, size=(300000))
C = A + B/1000000

C = np.sort(C)

A = C.astype(int)
B = C-A
hyperbola_solution(A,B)

1000000000